# A3.  Shotgun metagenomics - MAG recovery and Functional Prediction 

**Author: Anna Toidze**

In this notebook, I focus on analyzing shotgun metagenomic datasets. I have chosen 10 samples of 5 random patients present in Cohort 1 and Cohort 2 - namely, 'SRR23801859',  'SRR23801860',  'SRR23801866',  'SRR23801898',  'SRR23801908',  'SRR23801935',  'SRR23801941',  'SRR23801944',  'SRR23801959',  'SRR23801999'. Due to computational complexity, all of the following commands have been run the cluster (therefore, different usage of $data_dir than usual - commands run in the same directory, hence, `.`).  


**Exercise overview:**<br>
[0. Setup](#setup)<br>
[1. Sequence preprocessing](#seq)<br>
[2. Shotgun metagenome analysis](#moshpit)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.1 Genome assembly](#assembly)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2 Read mapping](#mapping)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3 Contig binning](#binning)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.4 MAG dereplication](#dereplication)<br>
[3. MAG abundance estimation](#moshpit)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1 MAG indexing](#indexing)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.2 Read mapping](#mapping)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.3 MAG length calculation](#feature-length)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.4 Abundance estimation](#abundance)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.5 Taxonomic classification of MAGs](#mag_classification)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.6 Taxonomic classification of reads](#read_classification)
[4. Functional annotation](#annotation)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[4.1 Ortholog search with Diamond](#diamond)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[4.2 Ortholog annotation](#eggnog)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[4.3 Annotation extraction](#extraction)<br>
[5. Troubleshooting](#trouble)<br>

<a id='setup'></a>

## 0. Setup

In [2]:
import pandas as pd
import qiime2 as q2
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

data_dir = 'w9_data'

In [3]:
# assigning variables throughout the notebook

# location of this week's data and all the results produced by this notebook
# - this should be a path relative to your working directory
raw_data_dir = "../data/raw"
data_dir = "../data/processed"
vis_dir  = "../results"

In [4]:
# setting seaborn formatting style
sns.set_style("white")
sns.set_theme(style="ticks", palette="pastel")

<a id='seq'></a>
## 1. Sequence preprocessing

In [5]:
meta = pd.read_csv(f"{raw_data_dir}/metadata.tsv", sep="\t")
meta.head()

,Sample_Name,Patient_ID,Stool_Consistency,Patient_Sex,Sample_Day,Recovery_Day,Cohort_Number
0,EG2580,P042,liquid,F,13,17.0,2
1,EG2559,P043,liquid,M,15,17.0,2
2,EG2537,P042,liquid,F,0,17.0,1
3,EG2518,P043,liquid,M,0,17.0,1
4,EG2490,P030,formed,F,0,NaN,1


In [7]:
wgs_ids = pd.read_csv(f"{raw_data_dir}/wgs_ids.txt", sep="\t")
wgs_ids.head()

,Run,AssayType,PatientID,Timepoint
0,SRR23801859,WGS,P027,35
1,SRR23801860,WGS,P014,14
2,SRR23801861,WGS,P043,0
3,SRR23801863,WGS,P039,0
4,SRR23801864,WGS,P028,0


In [12]:
merged_df = pd.merge(meta, wgs_ids, left_on=["Patient_ID", "Sample_Day"], right_on=["PatientID", "Timepoint"], how="inner")
merged_df  

,Sample_Name,Patient_ID,Stool_Consistency,Patient_Sex,Sample_Day,Recovery_Day,Cohort_Number,Run,AssayType,PatientID,Timepoint
0,EG2537,P042,liquid,F,0,17.0,1,SRR23801919,WGS,P042,0
1,EG2518,P043,liquid,M,0,17.0,1,SRR23801861,WGS,P043,0
2,EG0136,P027,formed,M,0,33.0,1,SRR23801959,WGS,P027,0
3,EG2149,P024,formed,F,0,37.0,1,SRR23801920,WGS,P024,0
4,EG2018,P014,semi-formed,F,14,16.0,2,SRR23801860,WGS,P014,14
5,EG1976,P014,formed,F,0,16.0,1,SRR23801941,WGS,P014,0
6,EG1940,P071,liquid,M,27,29.0,2,SRR23801928,WGS,P071,27
7,EG1939,P045,liquid,M,14,11.0,2,SRR23801868,WGS,P045,14
8,EG1878,P059,formed,M,22,20.0,2,SRR23801871,WGS,P059,22
9,EG1830,P008,semi-formed,F,0,NaN,1,SRR23801963,WGS,P008,0


Before the genome assembly, I have:

2. Downloaded the SRA Toolkit using conda - ```conda install bioconda::sra-tools```.
2. Extracting FASTQ Files: Use the fastq-dump utility from the SRA Toolkit to convert the SRA files into .fastq.gz format. ```fastq-dump --outdir fastq --gzip --skip-technical  --readids --read-filter pass --dumpbase --split-3 --clip SRRXXXXXX``` for each SRR file!
3. Imported the ```fastq.gz``` files into QIIME2.

In [ ]:
! fastq-dump --outdir fastq --gzip --skip-technical  --readids --read-filter pass --dumpbase --split-3 --clip SRRXXXXXX

In [ ]:
! qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path $raw_data_dir/manifest \
  --input-format PairedEndFastqManifestPhred33V2 \
  --output-path $data_dir/demux-paired-end.qza

Imported ../data/raw/manifest as PairedEndFastqManifestPhred33V2 to ../data/processed/demux-paired-end.qza


<a id='moshpit'></a>
## 2. Shotgun metagenome analysis

<a id='assembly'></a>
### 2.1 Genome assembly

In this step, I begin the process of recovering metagenome-assembled genomes (MAGs) by performing genome assembly. I am using MEGAHIT that takes short DNA sequencing reads, constructs a simplified De Bruijn graph, and produces longer contiguous sequences called contigs.


In [ ]:
qiime assembly assemble-megahit \
    --i-seqs  ./demux-paired-end.qza \
    --p-presets meta-sensitive \
    --p-num-cpu-threads 28 \
    --o-contigs ./contigs.qza

Error: QIIME 2 has no plugin/command named 'assembly'.


Once I’ve assembled the reads into contigs, I use QUAST to evaluate the quality of my assembly.

* N50: Measures the contiguity of the assembly. It’s the length of the contig where 50% of the genome is covered by contigs of that length or longer. Higher values indicate better assembly quality.
* L50: Indicates the number of contigs needed to cover 50% of the genome’s total length. Lower values are better.

In [ ]:
qiime assembly evaluate-contigs \
    --i-contigs ./contigs.qza \
    --p-threads 60 \
    --o-visualization ./contigs.qzv \
    --verbose

In [14]:
!qiime tools view $data_dir/contigs.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

<a id='mapping'></a>
### 2.2 Read mapping
Before assembling MAGs, I index the contigs from the assembly step and map the original reads to these contigs using the index. This mapping helps the contig binner group contigs from the same genome. I’ll now run the next two cells to perform contig indexing and read mapping.

In [ ]:
qiime assembly index-contigs \
    --i-contigs ./contigs.qza \
    --p-threads 60 \
    --p-seed 100 \
    --o-index ./contigs-index.qza

In [ ]:
qiime assembly map-reads \
    --i-index ./contigs-index.qza \
    --i-reads demux-paired-end.qza \
    --p-threads 60 \
    --p-seed 100 \
    --o-alignment-map ./reads-to-contigs-aln.qza

<a id='binning'></a>
### 2.3 Contig binning
Now I move on to contig binning, where I group contigs into bins based on their likely origin from different microbial species or strains in the community. For this, I’ll use MetaBAT 2, which assigns contigs to bins by analyzing tetranucleotide frequency and abundance (coverage) data.

In [ ]:
qiime moshpit bin-contigs-metabat \
    --i-contigs ./contigs.qza \
    --i-alignment-maps ./reads-to-contigs-aln.qza \
    --p-num-threads 60 \
    --p-seed 100 \
    --o-mags ./mags.qza \
    --o-contig-map ./contig-map.qza \
    --o-unbinned-contigs ./unbinned-contigs.qza

<a id='dereplication'></a>
### 2.4 MAG dereplication
The previous step generated a few artifacts:  
- `mags.qza`: the MAGs for each sample.  
- `contig-map.qza`: a mapping of MAG IDs to the contigs they include.  
- `unbinned-contigs.qza`: contigs that couldn’t be assigned to any MAG.  

From this point, I’ll focus on `mags.qza` and I dereplicate the MAG collection, considering that the 10 samples most likely share many genomes.

To dereplicate the MAGs, I:  
1. Compute hash sketches for each genome using [sourmash](https://sourmash.readthedocs.io/en/latest/), which creates compact representations of the genomes.  
2. Compare these sketches to generate a pairwise distance matrix.  
3. Dereplicate the genomes by clustering similar ones using a fixed similarity threshold, keeping the longest genome in each cluster.  

In [ ]:
qiime sourmash compute \
    --i-sequence-file ./mags.qza \
    --p-ksizes 105 \
    --p-scaled 100 \
    --o-min-hash-signature ./mags-hashes.qza

In [ ]:
qiime sourmash compare \
    --i-min-hash-signature ./mags-hashes.qza \
    --p-ksize 105 \
    --o-compare-output ./mags-dist.qza

In [ ]:
qiime moshpit dereplicate-mags \
    --i-mags ./mags.qza \
    --i-distance-matrix ./mags-dist.qza \
    --p-threshold 0.7 \
    --o-dereplicated-mags ./mags-derep.qza \
    --o-feature-table ./mags-table.qza

<a id='dereplication'></a>
### 2.5 MAG Quality Analysis

I performed the following quality analysis on the dereplicated set of MAGs due to computational limitations.

In [ ]:
! qiime moshpit evaluate-busco \
    --i-bins ./mags-derep.qza \
    --i-busco-db ./busco-db.qza \
    --p-lineage-dataset "bacteria_odb10" \
    --p-cpu 30 \
    --o-results-table ./busco-results.qza \
    --o-visualization ./mags-derep-qc.qzv


In [15]:
!qiime tools view $data_dir/mags-derep-qc.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

<a id='moshpit'></a>
## 3. MAG abundance estimation


In this section, I will estimate MAG abundance by mapping the original reads back to the recovered MAGs. I will then count the reads mapped to each genome, normalize these counts to genome lengths, and present the results using metrics such as RPKM or TPM.

<a id='indexing'></a>
### 3.1 MAG indexing
I first need to index our dereplicated MAGs. I can do it using the `index-derep-mags` action from the `q2-moshpit` plugin:

In [ ]:
# command you want to run
qiime assembly index-derep-mags \
    --i-mags ./mags-derep.qza \
    --p-threads 60 \
    --p-seed 100 \
    --o-index ./mags-derep-index.qza


<a id='mapping'></a>
### 3.2 Read mapping
Next, I will map the original reads to the dereplicated MAGs using the index generated in the previous step.

In [ ]:
qiime assembly map-reads \
    --i-index ./mags-derep-index.qza \
    --i-reads ./demux-paired-end.qza  \
    --p-threads 60 \
    --p-seed 100 \
    --o-alignment-map ./reads-to-mags-aln.qza

<a id='feature-length'></a>
### 3.3 MAG length calculation
I will calculate the length of each recovered MAG to normalize read counts based on genome length, using the get-feature-lengths action.

In [ ]:
qiime moshpit get-feature-lengths \
    --i-features ./mags-derep.qza \
    --o-lengths ./mags-derep-lengths.qza

<a id='abundance'></a>
### 3.4 Abundance estimation
I will estimate the abundance of our MAGs using RPKM (Reads Per Kilobase per Million mapped reads) as a metric, with a minimal mapping quality of 42 to include only perfectly mapped reads.

In [ ]:

qiime moshpit estimate-mag-abundance \
    --i-maps ./reads-to-mags-aln.qza \
    --i-mag-lengths ./mags-derep-lengths.qza \
    --p-metric "rpkm" \
    --p-min-mapq 42 \
    --p-threads 60 \
    --o-abundances ./mags-derep-ft.qza


<a id='mag_classification'></a>
### 3.5 Taxonomic classification of MAGs

I will perform the taxonomic classification of the assembled genomes using Kraken 2, a popular taxonomic classifier for metagenomic data. I will use a pre-built database containing k-mer profiles for archaeal, bacterial, viral, plasmid, and human sequences from NCBI's RefSeq database to compare the analyzed genomes to references.

First, I fetch the databases:

In [ ]:
!qiime moshpit build-kraken-db \
    --p-collection standard \
    --o-kraken2-database ./kraken_standard \
    --o-bracken-database ./bracken_standard \ 
    --verbose

I perform the classification:

In [ ]:
!qiime moshpit classify-kraken2 \
    --i-seqs ./mags-derep.qza \
    --i-kraken2-db ./kraken_standard \
    --p-threads 60 \
    --p-memory-mapping \
    --o-reports ./kraken2-reports-mags.qza \
    --o-hits ./kraken2-hits-mags.qza

I will convert the Kraken 2 report `FeatureData[Kraken2Report % Properties('mags')]`and MAG taxonomy output into a QIIME-compatible taxonomy format `FeatureData[Kraken2Output % Properties('mags')]`. The Kraken 2 report provides a tree-like representation of identified taxa, while the MAG taxonomy output lists MAGs with their corresponding identified taxa.

In [16]:
!qiime moshpit kraken2-to-mag-features \
    --i-reports ./kraken2-reports-mags.qza \
    --i-hits ./kraken2-hits-mags.qza \
    --o-taxonomy ./mags-taxonomy.qza

Error: QIIME 2 has no plugin/command named 'moshpit'.


In [19]:
# read in the taxonomy
mags_taxa = q2.Artifact.load(f"{data_dir}/mags-taxonomy.qza").view(pd.Series)
# read in the presence-absence table
mags_table = q2.Artifact.load(f"{data_dir}/mags-table.qza").view(pd.DataFrame)
# replace column names with the taxa
mags_table.columns = mags_taxa[mags_table.columns]

# transpose the table
mags_table = mags_table.T

for sample in mags_table.columns:
    print(f"Taxa identified in sample '{sample}':")
    for taxon in mags_table.loc[mags_table[sample] > 0, sample].index:
        print(f"  {taxon}")
    print()

Taxa identified in sample 'SRR23801859':
  d__Bacteria;k__Bacteria;p__Bacillota;c__Clostridia
  d__Bacteria;k__Bacteria;p__Actinomycetota;c__Actinomycetes;o__Bifidobacteriales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium adolescentis
  d__Bacteria;k__Bacteria;p__Bacillota;c__Clostridia;o__Eubacteriales;f__Oscillospiraceae
  d__Bacteria;k__Bacteria;p__Bacillota;c__Clostridia
  d__Bacteria;k__Bacteria
  d__Bacteria;k__Bacteria;p__Actinomycetota;c__Actinomycetes;o__Bifidobacteriales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium longum;ssp__Bifidobacterium longum subsp. longum
  d__Bacteria;k__Bacteria;p__Bacillota;c__Clostridia;o__Eubacteriales
  d__Bacteria;k__Bacteria;p__Bacillota;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus;s__Streptococcus thermophilus
  d__Bacteria;k__Bacteria;p__Actinomycetota;c__Coriobacteriia;o__Coriobacteriales;f__Coriobacteriaceae;g__Collinsella;s__Collinsella stercoris;ssp__Collinsella stercoris DSM 13279
  d__

Combine the previous results:

In [17]:
! qiime taxa barplot \
    --i-table ./mags-derep-ft.qza \
    --i-taxonomy ./mags-taxonomy.qza \
    --o-visualization ./mags-derep-barplot.qzv

^C

Aborted!


In [23]:
!qiime tools view $data_dir/mags-derep-barplot.qzv

Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.
Press the 'q' key, Control-C, or Control-D to quit. This view may no longer be accessible or work correctly after quitting.

<a id='annotation'></a>
## 4. Functional annotation
To discover genes and their functions within genomes, I will use the EggNOG-mapper. Functional annotation involves several steps: I will fetch the required databases, identify gene orthologs within each MAG, annotate each ortholog to determine its associated functions, and calculate the frequency of each annotation in the original samples.

<a id='diamond'></a>
### 4.1 Ortholog search with Diamond
I begin by fetching the Diamond protein database, followed by ortholog identification using the `eggnog-diamond-search` action:

In [ ]:
!qiime moshpit fetch-diamond-db --o-diamond-db eggnog-diamond/eggnog-diamond-db.qza --verbose >> eggnog-diamond/eggnoog-diamond-db.txt &

In [ ]:
!qiime moshpit eggnog-diamond-search \
    --i-sequences ./mags-derep.qza \
    --i-diamond-db ./eggnog-diamond/eggnog-diamond-db \
    --p-num-cpus 100 \
    --p-db-in-memory True \
    --o-eggnog-hits ./diamond-orthologs.qza \
    --o-table ./diamond-ft.qza


<a id='eggnog'></a>
### 4.2 Ortholog annotation
I use the EggNOG-mapper to annotate the orthologs found in the previous step. I fetch the required database and then run the `aggnog-annotate` action:

In [ ]:
! qiime moshpit fetch-diamond-db --o-diamond-db eggnog-diamond/eggnog-diamond-db.qza --verbose >> eggnog-diamond/eggnoog-diamond-db.txt &

In [ ]:
! qiime moshpit eggnog-annotate \
    --i-eggnog-hits ./diamond-orthologs.qza \
    --i-eggnog-db ./eggnog-annot-db.qza \
    --p-num-cpus 100 \
    --p-db-in-memory False \
    --o-ortholog-annotations ./eggnog-annotations.qza

<a id='extraction'></a>
### 4.3 Annotation extraction

I will extract specific annotations from the tables generated in the previous step, which include various annotation types for each ortholog, such as CAZymes, KEGG pathways, and COG categories. I will focus on COG categories and set a maximum E-value of 0.0001 to ensure only high-quality annotations are included.

In [ ]:
!qiime moshpit extract-annotations \
    --i-ortholog-annotations ./eggnog-annotations.qza \
    --p-annotation "cog" \
    --p-max-evalue 0.00001 \
    --o-annotation-frequency ./cog-ft.qza

!qiime moshpit multiply-tables \
    --i-table1 ./mags-derep-ft.qza \
    --i-table2 ./cog-ft.qza \
    --o-result-table ./mags-cog-ft.qza

! qiime taxa barplot \
    --i-table ./mags-cog-ft.qza \
    --o-visualization ./cog-barplot.qzv

<a id='trouble'></a>

## 5. Troubleshooting

Unfortunately, I had an issue with the databases, generating the followign error which I couldn't solve (especially, I couldn't figure out why because I just copied the commands, the respective directories for storing databases I have create before so it cannot be the reason):

(1/1) Invalid value for '--i-diamond-db': Archive does not contain a
  correctly formatted VERSION file.


Also, when downloading the eggnog-annot database on the latest QIIME shotgun release 2024-10, I would get a PlugInError when downloading it. Trying to solve this problem, took a long time and I couldn't find a solution... 

I also had way too many problems before, e.g. when  building kracken database, etc. It would be nice to have an official tutorial for shotgun sequencing, because the code from the provided notebooks is not working, at least for the newer versions of QIIME2.